## Instalar e importar módulos

In [1]:
#Beautiful Soup de bs4 y requests sirven para WebScraping (ya instalados con Anaconda)
#Selenium sirve para automatizar el navegador(desde Anaconda Prompt ejecutar: "pip install selenium")
#PyAutoGUI sirve para automatizar movimientos de mouse, clicks, teclado ("pip install pyautogui")
#pyperclip para trabajar con portapapeles ("pip install pyperclip")
#pandas para armar el dataframe, time para hacer pausas entre comandos

In [329]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import pyautogui as pg
import pyperclip as pc
import time

In [3]:
#Parsers de Scraping: lxml y html5lib (ya instalados con Anaconda)
#Webdriver de Chrome: Ir a "https://sites.google.com/a/chromium.org/chromedriver/downloads" para descargar un WebDriver
#para usarlo con Selenium  
#Revisar que sea la misma versión, el mío es ChromeDriver 83.0.4103.39 para Windows, para revisar la versión correcta
#ir a las opciones a la esquina superior derecha de Chrome > Help > About Google Chrome

## Entrar a PowerCampus > Clases Virtuales con Selenium

In [331]:
browser = webdriver.Chrome("C:\Program Files (x86)\chromedriver83.exe") #Acá puse el driver
browser.get("https://autoservicio2.up.edu.pe/ss/Home.aspx")
barra_user = browser.find_element_by_id("ctl00_ucUserLogin_lvLoginUser_ucLoginUser_lcLoginUser_UserName")
barra_password = browser.find_element_by_id("ctl00_ucUserLogin_lvLoginUser_ucLoginUser_lcLoginUser_Password")
barra_user.send_keys("jm.huamanpi") #Acá poner su usuario UP
barra_password.send_keys("831134DD") #Acá la contraseña
barra_password.send_keys(Keys.ENTER)
boton_clases = browser.find_element_by_link_text("Clases")
boton_clases.send_keys(Keys.ENTER)
boton_clasesvirtuales = browser.find_element_by_link_text("Clases Virtuales")
boton_clasesvirtuales.send_keys(Keys.ENTER)
pagina_base = browser.current_url
pagina_base

'https://autoservicio2.up.edu.pe/ss/UP/AdminClasesVirtuales/Forms/ListarClasesVirtuales.aspx'

## Navegar cursos y obtener links con Beautiful Soup o requests

### Videos de un curso

In [332]:
boton_desplegable = browser.find_element_by_id("select2-searchCursos-container")
boton_desplegable.click()
busqueda = browser.find_element_by_class_name("select2-search__field")
busqueda_desplegable = browser.find_element_by_xpath('//*[@id="ctl00_Html1"]/body/span/span/span[1]/input')
busqueda_desplegable.send_keys("econometria") #Nombre del curso
busqueda_desplegable.send_keys(Keys.ENTER)

In [229]:
# #Scraping
# pedido = requests.get(pagina_curso) #pedido GET request
# pedido
# html = pedido.content #su contenido
# soup = BeautifulSoup(html, "lxml") #soup es el objeto, lmxl es el parser
# #Ahora lo guardo en un archivo 'pedido.html'
# with open('pedido.html', "wb") as file:
#     file.write(soup.prettify("utf-8"))
# #Este pedido cambia con el Javascript así que no sirve, por eso lo sacaré con PyAutoGUI

In [337]:
#Scraping
time.sleep(1.8)
pg.hotkey('ctrl','shift','i')
time.sleep(1.5)
for i in range(0,2):
    pg.press('up')
pg.hotkey('shift','f10')
time.sleep(1)
for i in range(0,5):
    pg.press('down')
pg.press('right')
for i in range(0,2):
    pg.press('down')
pg.press('enter')
html = pc.paste()
html
soup = BeautifulSoup(html, "lxml") #soup es el objeto, lmxl es el parser
#Ahora lo guardo en un archivo 'html.html'
with open('html.html', "wb") as file:
    file.write(soup.prettify("utf-8"))
#time.sleep(1)
#pg.click(pg.locateCenterOnScreen('HTMLtag.png'), button='right')
#time.sleep(1)
#pg.click(pg.locateCenterOnScreen('Copy.png'))
#time.sleep(1)
#pg.click(pg.locateCenterOnScreen('Copyouterhtml.png'))

## Obtener nombres de cursos

In [338]:
cursoslista = soup.find('select', {"id":"searchCursos"})
cursostexto = cursoslista.contents
cursos=[] #Acá están los nombres de todos los cursos UP
for i in range(1,len(cursostexto)):
    cursos.append(cursostexto[i].text)

['Acto Jurídico(A)',
 'Administración de Operaciones(A)',
 'Administración de Operaciones(B)',
 'Administración de Operaciones(C)',
 'Administración de Operaciones(D)']

## Obtener cabeceras y datos

### Cabeceras

In [344]:
tabla = soup.find('table', {"class":"table","id":"tablaResultado"})
cabeceraslista = tabla.thead.tr
cabecerastexto = cabeceraslista.contents
cabeceras=[]
for i in range(1,len(cabecerastexto)-1,2):
    cabeceras.append(cabecerastexto[i].text)
cabeceras.append('Link2')
cabeceras

['Curso',
 'Sección',
 'Fecha',
 'Nombre de la sesión',
 'Duración',
 'Link',
 'Link2']

### Datos

In [341]:
datos = tabla.tbody
claseslista = datos.contents
curso=[]
seccion=[]
fecha=[]
nombreseccion=[]
duracion=[]
link=[] #Acá estarán los urls ca.bbcollab
link2=[] #Acá estarán los urls cloudfront
datostot=[curso,seccion,fecha,nombreseccion,duracion,link,link2]
# claseslista[#].find_all('td') ubica clase número #
# El .text obtiene el contenido de la etiqueta; el a.get('href') obtiene el url del link

for i in range(0,len(claseslista)):
    curso.append((claseslista[i].find_all('td')[0]).text)
    seccion.append((claseslista[i].find_all('td')[1]).text)
    fecha.append((claseslista[i].find_all('td')[2]).text)
    nombreseccion.append((claseslista[i].find_all('td')[3]).text)
    duracion.append((claseslista[i].find_all('td')[4]).text)
    link.append((claseslista[i].find_all('td')[5]).a.get('href'))

###  Obtener urls ca.bbcollab

In [342]:
#Pueden usar este código o el anterior
enlaces = browser.find_elements_by_xpath('.//a[text()="ingresar"]')
urls = [] #Acá están los urls
for n in enlaces:
    urls.append(n.get_attribute('href'))

### Obtener urls cloudfront

In [346]:
time.sleep(1.5)
for i in range(0,len(link)):
    browser.get(link[i])
    if i == 0:
        pg.hotkey('ctrl','shift','i') #Abrir las herramientas del desarrollador
        pg.hotkey('winleft','up') #Maximizar la ventana
        time.sleep(1.5)
        pg.click(pg.locateCenterOnScreen('Network.png'))
    time.sleep(10)
    pg.click(pg.locateCenterOnScreen('Media.png'),button='right')
    pg.click(pg.locateCenterOnScreen('Open.png'))
    browser.switch_to.window(browser.window_handles[1])
    link2.append(browser.current_url)
    pg.hotkey('ctrl','w')
    browser.switch_to.window(browser.window_handles[0])
link2
len(link2)

IndexError: list index out of range

## Marco de datos con pandas y exportar como csv

In [348]:
marco1 = pd.DataFrame()
for i in range(0,len(cabeceras)-1):
    marco1[cabeceras[i]]=datostot[i]
marco1

,Curso,Sección,Fecha,Nombre de la sesión,Duración,Link
0,Econometría I,C,2020-03-31,31.03.2020,1:41:02,https://ca.bbcollab.com/recording/a3254e84e7b1...
1,Econometría I,C,2020-04-02,02.04.2020,1:36:43,https://ca.bbcollab.com/recording/4844cc1ccd1c...
2,Econometría I,C,2020-04-04,PD 1,2:07:46,https://ca.bbcollab.com/recording/26932a15515d...
3,Econometría I,C,2020-04-07,Clase - Econometría I,1:38:00,https://ca.bbcollab.com/recording/2d9ee1652729...
4,Econometría I,C,2020-04-11,PD2,2:16:02,https://ca.bbcollab.com/recording/675f55b8e21a...
5,Econometría I,C,2020-04-14,Clase - Econometría I,1:33:50,https://ca.bbcollab.com/recording/c78d1b658ef0...
6,Econometría I,C,2020-04-16,Clase - Econometría I,1:42:35,https://ca.bbcollab.com/recording/1b366635f48a...
7,Econometría I,C,2020-04-18,PD3,2:01:24,https://ca.bbcollab.com/recording/9d56554171a2...
8,Econometría I,C,2020-04-21,Clase - Econometría I,1:44:35,https://ca.bbcollab.com/recording/9e9d3e2755bc...
9,Econometría I,C,2020-04-23,Clase - Econometría I,1:45:32,https://ca.bbcollab.com/recording/d191e0f4b544...


In [ ]:
marco1.to_csv("marco1.csv", index=True, header=True)